In [54]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import trimesh as tm
import boolean_marching_cubes as bmc

In [55]:
# example symmetry strings
sym_str = [["OO"], ["XX"], ["YY"], ["ZP"], ["ZN"]]

stencils = bmc.create_symmetry_stencils(sym_str)
# generate binary representation of all the possible cubes
l_bis = bmc.bi_cube_lattices()
# find all unique corner arrangements based on stencils
corner_profiles = bmc.extract_corner_profiles(stencils, l_bis)
# stack corner_profiles vertically
cp_stacked = np.vstack(corner_profiles)
# find the unique arrangements of corners
uniq_corner_arang = np.unique(cp_stacked, axis=0)
# construct lattices for all unique corner profiles
(corner_loc_lattices, corner_neigh_lattices) = bmc.profiles_to_lattices(uniq_corner_arang, stencils)

# save all design templates into lattice CSVs
templates_path = os.path.relpath('../data/bmc/bmc_templates1')
bmc.save_design_templates(corner_loc_lattices, corner_neigh_lattices, templates_path)

In [56]:
# load subtile meshes
Facade_meshes1 = []
for c in range(len(corner_loc_lattices)):
    corner_mesh_path = os.path.relpath('../data/bmc/facade_tile/t_' + f'{c:02}' + '.obj')
    corner_mesh = tm.load(corner_mesh_path)
    Facade_meshes1.append(corner_mesh)

tiles_meshes1 = bmc.construct_tile_meshes_old(Facade_meshes1, corner_profiles, uniq_corner_arang, corner_loc_lattices)
#tiles_meshes = construct_tile_meshes(subtile_meshes, corner_profiles, uniq_corner_arang, corner_loc_lattices)

In [57]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh
pv.set_plot_theme("document")
p = pv.Plotter(notebook=False)

base_lattice = l_bis[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit *0.5

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    mesh = tiles_meshes1[i]
    lattice = l_bis[i]

    # Add the data values to the cell data
    grid.cell_arrays["cube"] = lattice.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="cube")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=0.2, show_scalar_bar=False, color="C86F64")

    # adding the meshes
    p.add_mesh(tri_to_pv(mesh), color='#463E5F', name="sphere")

    return

p.add_slider_widget(create_mesh, [0, len(tiles_meshes1)], title='Tiles', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))

p.show(use_ipyvtk=True)

  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_21896/3015785431.py", line 28, in create_mesh
    mesh = tiles_meshes1[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_21896/3015785431.py", line 28, in create_mesh
    mesh = tiles_meshes1[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_21896/3015785431.py", line 28, in create_mesh
    mesh = tiles_meshes1[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_21896/3015785431.py", line 28, in create_mesh
    mesh = tiles_meshes1[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_21896/3015785431.py", line 28, in create_mesh
    mesh = tiles_meshes1[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppData\Local\Temp/ipykernel_21896/3015785431.py", line 28, in create_mesh
    mesh = tiles_meshes1[i]
IndexError: list index out of range
  File "C:\Users\sk928\AppDa

[(1.9318516525781368, 1.9318516525781368, 1.9318516525781368),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 1.0)]

In [58]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope_low.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)
print(envelope_lattice.shape)

padded_env_arr = np.pad(envelope_lattice, 1, mode="constant", constant_values=False)
padded_minbound = envelope_lattice.minbound - envelope_lattice.unit
envelope_lattice = tg.to_lattice(padded_env_arr, minbound=padded_minbound, unit=envelope_lattice.unit)
print(envelope_lattice.shape)
cube_lattice = envelope_lattice.boolean_marching_cubes()

(6, 6, 5)
(8, 8, 7)


In [59]:
#save facade tile mesh
def save_tile_meshes(tiles_meshes1, l_bis, tiles_path):
    pow_two = 2 ** np.arange(7, -1, -1)
    for tile_mesh, l in zip(tiles_meshes1, l_bis):
        l = np.roll(l, [1, 1, 1], [0, 1, 2])
        c_id = np.sum(l.flatten(order='F') * pow_two)
        tile_mesh_path = os.path.join(tiles_path, 't_' + f'{c_id:03}' + '.obj')
        with open(tile_mesh_path, 'w') as file:
            file.write(tm.exchange.obj.export_obj(tile_mesh))

tiles_path = os.path.relpath('../data/bmc/BMC_DUMMY_256_2')
save_tile_meshes(tiles_meshes1, l_bis, tiles_path)

In [60]:
def marching_cube_mesh_2(cube_lattice, tiles_path):

    # extract cube indices
    cube_ind = np.transpose(np.indices(cube_lattice.shape),
                            (1, 2, 3, 0)).reshape(-1, 3)
    # extract cube positions
    cube_pos = (cube_ind + 0.5) * cube_lattice.unit + cube_lattice.minbound
    # extract cube tid
    cube_tid = cube_lattice.ravel()
    # remove the cube position and tid where tid is 0
    filled_cube_pos = cube_pos[cube_tid > 0]
    filled_cube_tid = cube_tid[cube_tid > 0]
    # load tiles
    # set how manu tileset
    tilesetnumber = 1
    tiles = [0]
    for i in range(1, 256*tilesetnumber):
        tile_path = os.path.join(tiles_path, 't_' + f'{i:03}' + '.obj')
        tile = tm.load(tile_path)
        tile.vertices *= cube_lattice.unit
        tiles.append(tile)
    last_v_count = 0
    vertice_list = []
    face_list = []
    # place tiles
    for i in range(1, filled_cube_tid.size):
        # extract current tile
        tile = tiles[filled_cube_tid[i]]
        # append the vertices
        vertice_list.append(tile.vertices + filled_cube_pos[i])
        face_list.append(tile.faces + last_v_count)
        last_v_count += len(tile.vertices)
    vs = []
    fs = []
    # if len(vertice_list):
    vs = np.vstack(vertice_list)
    fs = np.vstack(face_list)
    tile_mesh = tm.Trimesh(vs, fs)
    return tile_mesh

bmc_facademesh = marching_cube_mesh_2(cube_lattice,tiles_path)


In [61]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh
pv.set_plot_theme("document")
# initiating the plotter
p = pv.Plotter(notebook=False)

# # adding the meshes
p.add_mesh(tri_to_pv(bmc_facademesh), color='#638275', name="sphere", opacity=1)
# # fast visualization of the lattice
p = envelope_lattice.fast_vis(p)
#p.add_mesh(tri_to_pv(context_mesh), style='surface',opacity=0.3)
#p.add_mesh(tri_to_pv(bmc_facademesh), style ='surface',opacity=1)
# plotting
#cpos = [(719.962336021059, 207.0623203157632, 62.36825901611057),
#  (646.0, 330.0, 8.0),
#  (-0.17822556079522067, 0.2942981909318281, 0.9389484673258075)]
 
#p.camera_position = cpos
p.show(use_ipyvtk=True)

[(334.02596918053655, 316.7459699244025, 264.04196706719426),
 (95.04000186920166, 77.76000261306763, 25.055999755859375),
 (0.0, 0.0, 1.0)]